# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Project 4: West-Nile-Virus-Prediction

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pandas_profiling as pdpro
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from datetime import datetime
from sklearn import ensemble, preprocessing

### Read in data, view dtypes, etc.

In [2]:
# Load dataset 
train = pd.read_csv('../assets/train.csv')
test = pd.read_csv('../assets/test.csv')
sample = pd.read_csv('../assets/sampleSubmission.csv')
weather = pd.read_csv('../assets/weather.csv')

###### 1. train and test data cleaning

In [3]:
#profile_train= train.profile_report(title='Pandas Profiling Report', plot={'histogram': {'bins': 8}})
#profile_train.to_file(output_file="output_train.html")

In [4]:
#profile_test = test.profile_report(title='Pandas Profiling Report', plot={'histogram': {'bins': 8}})
#profile_test.to_file(output_file="output_test.html")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
Date                      10506 non-null object
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 985.1+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
Id                        116293 non-null int64
Date                      116293 non-null object
Address                   116293 non-null object
Species                   116293 non-null object
Block                     116293 non-null int64
Street                    116293 non-null object
Trap                      116293 non-null object
AddressNumberAndStreet    116293 non-null object
Latitude                  116293 non-null float64
Longitude                 116293 non-null float64
AddressAccuracy           116293 non-null int64
dtypes: float64(2), int64(3), object(6)
memory usage: 9.8+ MB


In [7]:
train.Species.unique()

array(['CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX PIPIENS',
       'CULEX SALINARIUS', 'CULEX TERRITANS', 'CULEX TARSALIS',
       'CULEX ERRATICUS'], dtype=object)

In [8]:
test.Species.unique()

array(['CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX PIPIENS',
       'CULEX SALINARIUS', 'CULEX TERRITANS', 'CULEX TARSALIS',
       'UNSPECIFIED CULEX', 'CULEX ERRATICUS'], dtype=object)

In [9]:
# Functions to extract month and day from dataset
'''def create_month(x):
    return x.split('-')[1]

def create_day(x):
    return x.split('-')[2]

train['month'] = train.Date.apply(create_month)
train['day'] = train.Date.apply(create_day)
test['month'] = test.Date.apply(create_month)
test['day'] = test.Date.apply(create_day)'''

"def create_month(x):\n    return x.split('-')[1]\n\ndef create_day(x):\n    return x.split('-')[2]\n\ntrain['month'] = train.Date.apply(create_month)\ntrain['day'] = train.Date.apply(create_day)\ntest['month'] = test.Date.apply(create_month)\ntest['day'] = test.Date.apply(create_day)"

In [10]:
#train = pd.get_dummies(train, columns=['Species'],drop_first=True)
#test = pd.get_dummies(test, columns=['Species'],drop_first=True)

In [11]:
train['Date'] = pd.to_datetime(train.Date)
test['Date'] = pd.to_datetime(test.Date)

In [12]:
'''# Add integer latitude/longitude columns
train['Lat_int'] = train.Latitude.apply(int)
train['Long_int'] = train.Longitude.apply(int)
test['Lat_int'] = test.Latitude.apply(int)
test['Long_int'] = test.Longitude.apply(int)'''

"# Add integer latitude/longitude columns\ntrain['Lat_int'] = train.Latitude.apply(int)\ntrain['Long_int'] = train.Longitude.apply(int)\ntest['Lat_int'] = test.Latitude.apply(int)\ntest['Long_int'] = test.Longitude.apply(int)"

In [13]:
# Get labels(boolean) as target
labels = train.WnvPresent.values

In [14]:
# drop address columns(NumMosquitos has no correlation with target wvnpresent)
train = train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos','Block',], axis = 1)
test = test.drop(['Id', 'Address', 'AddressNumberAndStreet','Block',], axis = 1)

###### 2. weather data cleaning

In [15]:
#profile_weather= weather.profile_report(title='Pandas Profiling Report', plot={'histogram': {'bins': 8}})
#profile_weather.to_file(output_file="output_weather.html")

In [16]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


In [17]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


remove'M','T','-'

In [18]:
# 'M' in weather data is a missing value; convert all instances to np.nan
weather.replace(to_replace='M', value=np.nan, inplace=True)

In [19]:
# Sunrise and sunset values are omitted for the second reading of each day; this will allow us to backfill them all
# First convert empty cells to np.nan
weather.replace(to_replace='-', value=np.nan, inplace=True)

In [20]:
# Codesum column has many empty values; convert all empty cells to np.nan
weather.replace(to_replace=' ', value=np.nan, inplace=True)

In [21]:
weather.replace(to_replace='T', value=0, inplace=True)
weather.replace(to_replace=' T', value=0, inplace=True)
weather.replace(to_replace='  T', value=0, inplace=True)

convert column type to numeric

In [22]:
weather[['WetBulb','Heat','Cool','PrecipTotal','StnPressure','Tavg','Depth','AvgSpeed','SeaLevel']] = weather[[
    'WetBulb','Heat','Cool','PrecipTotal','StnPressure','Tavg','Depth','AvgSpeed','SeaLevel']].apply(pd.to_numeric, errors ='ignore')

In [23]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2933 non-null float64
Depart         1472 non-null object
DewPoint       2944 non-null int64
WetBulb        2940 non-null float64
Heat           2933 non-null float64
Cool           2933 non-null float64
Sunrise        1472 non-null object
Sunset         1472 non-null object
CodeSum        1335 non-null object
Depth          1472 non-null float64
Water1         0 non-null float64
SnowFall       1472 non-null object
PrecipTotal    2942 non-null float64
StnPressure    2940 non-null float64
SeaLevel       2935 non-null float64
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2941 non-null float64
dtypes: float64(11), int64(5), object(6)
memory usage: 506.1+ KB


drop the not meaning for columns or information repetition 

In [24]:
weather.drop(['Sunrise','Sunset','CodeSum','Water1','Depart'],axis=1,inplace=True)

In [25]:
weather.Station.unique()

array([1, 2])

In [26]:
weather.shape

(2944, 17)

In [27]:
# Split station 1 and 2 and join horizontally. if use dummy, will still keep the same date with 2 rows for 2 stations
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [28]:
weather.shape

(1472, 31)

In [29]:
# Now identify number of missing values in each column
weather.isnull().sum()

Date                0
Tmax_x              0
Tmin_x              0
Tavg_x              0
DewPoint_x          0
WetBulb_x           3
Heat_x              0
Cool_x              0
Depth_x             0
SnowFall_x          0
PrecipTotal_x       0
StnPressure_x       2
SeaLevel_x          5
ResultSpeed_x       0
ResultDir_x         0
AvgSpeed_x          0
Tmax_y              0
Tmin_y              0
Tavg_y             11
DewPoint_y          0
WetBulb_y           1
Heat_y             11
Cool_y             11
Depth_y          1472
SnowFall_y       1472
PrecipTotal_y       2
StnPressure_y       2
SeaLevel_y          4
ResultSpeed_y       0
ResultDir_y         0
AvgSpeed_y          3
dtype: int64

further drop unuseful columns of weather

In [30]:
weather.Depth_x.unique()

array([0.])

In [31]:
weather.Depth_y=1
weather.Depth_y.unique()

array([1])

In [32]:
weather.SnowFall_x.unique()

array(['0.0', 0, '0.1'], dtype=object)

In [33]:
weather.drop(['SnowFall_x','SnowFall_y'],axis=1,inplace=True)

In [34]:
weather=weather.dropna()

In [35]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1447 entries, 0 to 1471
Data columns (total 29 columns):
Date             1447 non-null object
Tmax_x           1447 non-null int64
Tmin_x           1447 non-null int64
Tavg_x           1447 non-null float64
DewPoint_x       1447 non-null int64
WetBulb_x        1447 non-null float64
Heat_x           1447 non-null float64
Cool_x           1447 non-null float64
Depth_x          1447 non-null float64
PrecipTotal_x    1447 non-null float64
StnPressure_x    1447 non-null float64
SeaLevel_x       1447 non-null float64
ResultSpeed_x    1447 non-null float64
ResultDir_x      1447 non-null int64
AvgSpeed_x       1447 non-null float64
Tmax_y           1447 non-null int64
Tmin_y           1447 non-null int64
Tavg_y           1447 non-null float64
DewPoint_y       1447 non-null int64
WetBulb_y        1447 non-null float64
Heat_y           1447 non-null float64
Cool_y           1447 non-null float64
Depth_y          1447 non-null int64
PrecipTotal_y

In [36]:
weather['Date'] = pd.to_datetime(weather.Date)

###### 3. merge train and weather, test and weather

In [37]:
train['Date'].dt.year.unique()

array([2007, 2009, 2011, 2013])

In [38]:
test['Date'].dt.year.unique()

array([2008, 2010, 2012, 2014])

In [39]:
weather['Date'].dt.year.unique()

array([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014])

In [40]:
# Merge with weather data
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')
train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)

In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10367 entries, 0 to 10366
Data columns (total 34 columns):
Species            10367 non-null object
Street             10367 non-null object
Trap               10367 non-null object
Latitude           10367 non-null float64
Longitude          10367 non-null float64
AddressAccuracy    10367 non-null int64
Tmax_x             10367 non-null int64
Tmin_x             10367 non-null int64
Tavg_x             10367 non-null float64
DewPoint_x         10367 non-null int64
WetBulb_x          10367 non-null float64
Heat_x             10367 non-null float64
Cool_x             10367 non-null float64
Depth_x            10367 non-null float64
PrecipTotal_x      10367 non-null float64
StnPressure_x      10367 non-null float64
SeaLevel_x         10367 non-null float64
ResultSpeed_x      10367 non-null float64
ResultDir_x        10367 non-null int64
AvgSpeed_x         10367 non-null float64
Tmax_y             10367 non-null int64
Tmin_y             10367

In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 34 columns):
Species            116293 non-null object
Street             116293 non-null object
Trap               116293 non-null object
Latitude           116293 non-null float64
Longitude          116293 non-null float64
AddressAccuracy    116293 non-null int64
Tmax_x             116293 non-null int64
Tmin_x             116293 non-null int64
Tavg_x             116293 non-null float64
DewPoint_x         116293 non-null int64
WetBulb_x          116293 non-null float64
Heat_x             116293 non-null float64
Cool_x             116293 non-null float64
Depth_x            116293 non-null float64
PrecipTotal_x      116293 non-null float64
StnPressure_x      116293 non-null float64
SeaLevel_x         116293 non-null float64
ResultSpeed_x      116293 non-null float64
ResultDir_x        116293 non-null int64
AvgSpeed_x         116293 non-null float64
Tmax_y             116293 non-null int64
T

In [43]:

# Convert categorical data to numbers
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

lbl.fit(list(train['Street'].values) + list(test['Street'].values))
train['Street'] = lbl.transform(train['Street'].values)
test['Street'] = lbl.transform(test['Street'].values)

lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

In [44]:
train.head()

,Species,Street,Trap,Latitude,Longitude,AddressAccuracy,Tmax_x,Tmin_x,Tavg_x,DewPoint_x,...,WetBulb_y,Heat_y,Cool_y,Depth_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2,36,1,41.954690,-87.800991,9,88,60,74.0,58,...,66.0,0.0,12.0,1,0.0,29.44,30.09,5.8,16,7.4
1,3,36,1,41.954690,-87.800991,9,88,60,74.0,58,...,66.0,0.0,12.0,1,0.0,29.44,30.09,5.8,16,7.4
2,3,30,8,41.994991,-87.769279,9,88,60,74.0,58,...,66.0,0.0,12.0,1,0.0,29.44,30.09,5.8,16,7.4
3,2,120,15,41.974089,-87.824812,8,88,60,74.0,58,...,66.0,0.0,12.0,1,0.0,29.44,30.09,5.8,16,7.4
4,3,120,15,41.974089,-87.824812,8,88,60,74.0,58,...,66.0,0.0,12.0,1,0.0,29.44,30.09,5.8,16,7.4


In [45]:
test.head()

,Species,Street,Trap,Latitude,Longitude,AddressAccuracy,Tmax_x,Tmin_x,Tavg_x,DewPoint_x,...,WetBulb_y,Heat_y,Cool_y,Depth_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2,36,1,41.95469,-87.800991,9,86,61,74.0,56,...,64.0,0.0,11.0,1,0.0,29.34,29.97,9.4,18,10.4
1,3,36,1,41.95469,-87.800991,9,86,61,74.0,56,...,64.0,0.0,11.0,1,0.0,29.34,29.97,9.4,18,10.4
2,1,36,1,41.95469,-87.800991,9,86,61,74.0,56,...,64.0,0.0,11.0,1,0.0,29.34,29.97,9.4,18,10.4
3,4,36,1,41.95469,-87.800991,9,86,61,74.0,56,...,64.0,0.0,11.0,1,0.0,29.34,29.97,9.4,18,10.4
4,6,36,1,41.95469,-87.800991,9,86,61,74.0,56,...,64.0,0.0,11.0,1,0.0,29.34,29.97,9.4,18,10.4


In [46]:
train.to_csv('../final_train_test/train.csv',encoding='utf-8')
test.to_csv('../final_train_test/test.csv',encoding='utf-8')